## <b> Лабораторна робота №2
"Метрики"\
Виконала: Челюскіна Юлія КМ-21 (2-га підгрупа)

---

### Початкові дані (1)

In [3]:
import pandas as pd
df  = pd.read_csv("KM-21-2.csv", delimiter=",")
df



,GT,Model_1_0,Model_2_1
0,1,0.014745,0.996526
1,0,0.999735,0.003289
2,0,0.999786,0.125449
3,0,0.999695,0.000858
4,1,0.022115,0.786606
...,...,...,...
4395,0,0.999753,0.125449
4396,0,0.999777,0.125449
4397,0,0.909023,0.159557
4398,0,0.999737,0.125999


### Збалансованість (2)


In [4]:
df.describe()

,GT,Model_1_0,Model_2_1
count,4400.000000,4400.000000,4400.000000
mean,0.500000,0.502790,0.487270
std,0.500057,0.487012,0.435648
min,0.000000,0.000000,0.000000
25%,0.000000,0.009073,0.054026
50%,0.500000,0.216863,0.613696
75%,1.000000,0.999784,0.990292
max,1.000000,0.999973,0.998421


In [5]:
#Визначити збалансованість набору даних. Вивести кількість об’єктів кожного класу
df["GT"].value_counts()

GT
1    2200
0    2200
Name: count, dtype: int64

### Метрики

* Матриця помилок

|                    | Predicted Positive (1) | Predicted Negative (0) |
|--------------------|----------------------|----------------------|
| **Actual Positive (1)** | TP                   | FP                   |
| **Actual Negative (0)** | FN                   | TN                   |


* Accuracy = ( TP+TN  ) / ( TP+TN+FP+FN ) \
*Limitations of Accuracy:* Imbalanced Datasets,  The cost of false positives and negatives

* Precision = TP / ( TP+FP )\
(скільки з названих позитивними є справді позитивними)\
зменшення хибних тривог (1 рід помилок)

* Recall = TP / TP + FN\
TPR (скільки з позитивних було названо як позитивні)\
зменшення пропуску цілі (2 рід помилок)

* F-Score = 2 × ( ( Precision * Recall ) / ( Precision + Recall ) ) \
Гармонічне середнє точності та повноти

In [29]:
import sklearn.metrics as m
import matplotlib.pyplot as plt 

def model_metrics(res, goal):
    

    thresholds = [round((0.05+x*0.15), 2) for x in range(7)]
    accuracy = []
    precision = []
    bal_accuracy = []
    recall = []
    f_score = []
    matthews = []
    J = []
    roc_auc = []
    rp_auc = []
    
    print("Thresholds: ", thresholds)   
    for i in thresholds:
        if goal == 0:
            real = [x==0 for x in df["GT"]]
            pred = ( res > i).astype(int)
        else:
            real = df["GT"]
            pred = ( res > i).astype(int)
        conf_matr = m.confusion_matrix(real, pred)
        TP = conf_matr[0][0]
        FP = conf_matr[0][1]
        TN = conf_matr[1][1]
        FN = conf_matr[1][0]
        accuracy.append(m.accuracy_score(real, pred))
        precision.append(m.precision_score(real, pred))
        bal_accuracy.append(m.balanced_accuracy_score(real, pred))
        recall.append(m.recall_score(real, pred))
        f_score.append(m.f1_score(real, pred))
        matthews.append(m.matthews_corrcoef(real, pred))
        conf_matr = m.confusion_matrix(real, pred)
        J.append(TP/(TP+FN) + TN/(TN+FP) - 1)
        roc_auc.append(m.roc_auc_score(real, pred))
        p, r, _ = m.precision_recall_curve(real, pred)
        rp_auc.append(m.auc(r, p))

    print(f"Accuracy: {[round(a, 3) for a in accuracy]}")
    print(f"Precision: {[round(p, 3) for p in precision]}")
    print(f"Balanced Accuracy: {[round(b, 3) for b in bal_accuracy]}")
    print(f"Recall: {[round(r, 3) for r in recall]}")
    print(f"F-Score: {[round(f, 3) for f in f_score]}")
    print(f"Matthews Corr Coef: {[round(m, 3) for m in matthews]}")
    print(f"Youden's J Statistic: {[round(j, 3) for j in J]}")
    print(f"ROC AUC: {[round(r, 3) for r in roc_auc]}")
    print(f"Precision-Recall AUC: {[round(rp, 3) for rp in rp_auc]}")
    

    

In [30]:
model_metrics(df["Model_1_0"], 0)

Thresholds:  [0.05, 0.2, 0.35, 0.5, 0.65, 0.8, 0.95]
Accuracy: [0.939, 0.986, 0.991, 0.99, 0.989, 0.986, 0.978]
Precision: [0.894, 0.984, 0.995, 0.995, 0.996, 0.996, 0.999]
Balanced Accuracy: [0.939, 0.986, 0.991, 0.99, 0.989, 0.986, 0.978]
Recall: [0.995, 0.989, 0.986, 0.984, 0.981, 0.975, 0.957]
F-Score: [0.942, 0.986, 0.991, 0.99, 0.989, 0.986, 0.977]
Matthews Corr Coef: [0.883, 0.973, 0.981, 0.98, 0.977, 0.972, 0.957]
Youden's J Statistic: [0.889, 0.973, 0.981, 0.98, 0.977, 0.972, 0.958]
ROC AUC: [0.939, 0.986, 0.991, 0.99, 0.989, 0.986, 0.978]
Precision-Recall AUC: [0.946, 0.989, 0.994, 0.994, 0.993, 0.992, 0.989]


In [23]:
model_metrics(df["Model_2_1"], 1)

Thresholds:  [0.05, 0.2, 0.35, 0.5, 0.65]
Accuracy: [0.747, 0.991, 0.991, 0.991, 0.985]
Precision: [0.664, 0.987, 0.987, 0.988, 0.99]
Balanced Accuracy: [0.747, 0.991, 0.991, 0.991, 0.985]
Recall: [1.0, 0.995, 0.995, 0.995, 0.98]
F-Score: [0.798, 0.991, 0.991, 0.991, 0.985]
Matthews Corr Coef: [0.573, 0.982, 0.983, 0.983, 0.97]
Youden's J Statistic: [0.663, 0.982, 0.983, 0.983, 0.97]
ROC AUC: [0.747, 0.991, 0.991, 0.991, 0.985]
Precision-Recall AUC: [0.832, 0.992, 0.993, 0.993, 0.99]


Використані джерела:
* https://www.geeksforgeeks.org/sklearn-classification-metrics/
* https://www.kaggle.com/code/pythonafroz/evaluation-metrics-used-in-machine-learning